In [19]:
#%%
"""
Created on Thu Nov 27 2018
Pricing of European Call and Put options wit the COS method
@author: Lech A. Grzelak
"""
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import time

def CallPutOptionPriceCOSMthd(cf,CP,S0,r,tau,K,N,L,a,b):
    # cf   - characteristic function as a functon, in the book denoted as \varphi
    # CP   - C for call and P for put
    # S0   - Initial stock price
    # r    - interest rate (constant)
    # tau  - time to maturity
    # K    - list of strikes
    # N    - Number of expansion terms
    # L    - size of truncation domain (typ.:L=8 or L=10)  
        
    # reshape K to a column vector
    K = np.array(K).reshape([len(K),1])
    
    #assigning i=sqrt(-1)
    i = np.complex(0.0,1.0) 
    
    x0 = np.log(S0 / K)   
    
    # truncation domain
#     a = 0.0 - L * np.sqrt(tau)
#     b = 0.0 + L * np.sqrt(tau)
    
    # sumation from k = 0 to k=N-1
    k = np.linspace(0,N-1,N).reshape([N,1])  
    u = k * np.pi / (b - a);  

    # Determine coefficients for Put Prices  
    H_k = CallPutCoefficients(CP,a,b,k)
       
    mat = np.exp(i * np.outer((x0 - a) , u))
    print(mat)

    temp = cf(u) * H_k 
    temp[0] = 0.5 * temp[0]    
    print(cf(u))
#     print("======")
#     print(H_k)
#     print("======")
#     print(temp)
    
    value = np.exp(-r * tau) * K * np.real(mat.dot(temp))
         
    return value

""" 
Determine coefficients for Put Prices 
"""
def CallPutCoefficients(CP,a,b,k):
    if str(CP).lower()=="c" or str(CP).lower()=="1":                  
        c = 0.0
        d = b
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        if a < b and b < 0.0:
            H_k = np.zeros([len(k),1])
        else:
            H_k      = 2.0 / (b - a) * (Chi_k - Psi_k)  
        
    elif str(CP).lower()=="p" or str(CP).lower()=="-1":
        c = a
        d = 0.0
        coef = Chi_Psi(a,b,c,d,k)
        Chi_k = coef["chi"]
        Psi_k = coef["psi"]
        H_k      = 2.0 / (b - a) * (- Chi_k + Psi_k)               
    
    return H_k    

def Chi_Psi(a,b,c,d,k):
    psi = np.sin(k * np.pi * (d - a) / (b - a)) - np.sin(k * np.pi * (c - a)/(b - a))
    psi[1:] = psi[1:] * (b - a) / (k[1:] * np.pi)
    psi[0] = d - c
    
    chi = 1.0 / (1.0 + np.power((k * np.pi / (b - a)) , 2.0)) 
    expr1 = np.cos(k * np.pi * (d - a)/(b - a)) * np.exp(d)  - np.cos(k * np.pi 
                  * (c - a) / (b - a)) * np.exp(c)
    expr2 = k * np.pi / (b - a) * np.sin(k * np.pi * 
                        (d - a) / (b - a))   - k * np.pi / (b - a) * np.sin(k 
                        * np.pi * (c - a) / (b - a)) * np.exp(c)
    chi = chi * (expr1 + expr2)
    
    value = {"chi":chi,"psi":psi }
    return value
    

def BS_Call_Option_Price(CP,S_0,K,sigma,tau,r):
    #Black-Scholes Call option price
    cp = str(CP).lower()
    K = np.array(K).reshape([len(K),1])
    d1    = (np.log(S_0 / K) + (r + 0.5 * np.power(sigma,2.0)) 
    * tau) / float(sigma * np.sqrt(tau))
    d2    = d1 - sigma * np.sqrt(tau)
    if cp == "c" or cp == "1":
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf(d2) * K * np.exp(-r * tau)
    elif cp == "p" or cp =="-1":
        value = st.norm.cdf(-d2) * K * np.exp(-r * tau) - st.norm.cdf(-d1)*S_0
    return value

In [20]:
def mainCalculation():
    i = np.complex(0.0,1.0)
    
    CP = "c"
    S0 = 1900.0
    r = 0.013
    tau = 0.25
    sigma = 0.36
    K = [2000.0, 2100.0, 2200.0]
    N = 2**12
    L = 10
    cf = lambda u: np.exp((r - 0.5 * np.power(sigma,2.0)) * i * u * tau - 0.5 
                          * np.power(sigma, 2.0) * np.power(u, 2.0) * tau)
    ab = [[-1, 1], [-4, 4], [-8, 8], [-12, 12]]
    for ran in ab:
        val_COS = CallPutOptionPriceCOSMthd(cf,CP,S0,r,tau,K,N,L, ran[0],ran[1])
#         print(val_COS)
    
mainCalculation()

[[ 1.        +0.j          0.08048417+0.99675589j -0.9870446 +0.16044614j
  ...  0.08870562-0.99605789j  0.99996596+0.00825095j
   0.07225725+0.99738603j]
 [ 1.        +0.j          0.15656395+0.98766782j -0.95097546+0.30926634j
  ...  0.53370856-0.84566848j  0.91879906+0.39472558j
  -0.24600694+0.96926807j]
 [ 1.        +0.j          0.22825423+0.97360156j -0.89580001+0.44445735j
  ...  0.07535792+0.99715655j -0.95363241+0.30097379j
  -0.51069919-0.85975947j]]
[[ 1.        +0.j        ]
 [ 0.96061766-0.0195434j ]
 [ 0.85153438-0.03466263j]
 ...
 [-0.        -0.j        ]
 [-0.        -0.j        ]
 [-0.        -0.j        ]]
[[ 1.        +0.j          0.02014147+0.99979714j -0.99918864+0.04027476j
  ...  0.69123202+0.72263289j -0.70856387+0.70564669j
  -0.71977505-0.69420737j]
 [ 1.        +0.j          0.03929256+0.99922775j -0.99691219+0.07852444j
  ... -0.60092127-0.79930822j  0.77507921-0.63186408j
   0.66183097+0.7496531j ]
 [ 1.        +0.j          0.05753925+0.99834324j -0.993

/var/folders/m4/52jcfz5d7tldnd1yw00622fm0000gn/T/ipykernel_46318/1033533780.py:2: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  i = np.complex(0.0,1.0)
/var/folders/m4/52jcfz5d7tldnd1yw00622fm0000gn/T/ipykernel_46318/3148615317.py:26: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  i = np.complex(0.0,1.0)
